# Limestone addition to the cement recipe

<p class="acknowledgement">Written by Svetlana Kyas (ETH Zurich) on Feb 4th, 2022</p>

This tutorial demonstrates how Reaktoro can be used for performing realistic modeling of the limestone addition
to the cement recipe and effects this mixing causes.

```{note}
If your main interest is in computing thermodynamic properties, rather than chemical equilibrium and
kinetics modeling, you may want to check [ThermoFun](https://thermohub.org/thermofun/thermofun/),
which is an excellent project dedicated to this task.

This tutorial is joint work with Dan Miron, PSI.
```

The considered model uses a **thermofun** database `cemdata18` based on [Cemdata](https://www.empa.ch/web/s308/thermodynamic-data),
the thermodynamic data for hydrated solids in the Portland cement system (CaO-Al2O3-SiO2-CaSO4-CaCO3-Fe2O3-MgO-H2O).
The resulting chemical system includes an aqueous phase, minerals, and four solid phases.

In [ ]:
from reaktoro import *

# Define Thermofun database
db = ThermoFunDatabase("cemdata18")

# Define aqueous solution phase with Debye-Huckel activity model
solution = AqueousPhase(speciate("H O K Na S Si Ca Mg Al C Cl"))

# Set up a and b parameters for the ionic species (KOH, b = 0.123, a = 3.67)
params = ActivityModelDebyeHuckelParams()
params.aiondefault = 3.67
params.biondefault = 0.123
params.bneutraldefault = 0.123

solution.setActivityModel(ActivityModelDebyeHuckel(params))

# Define minerals phases
minerals = MineralPhases("Cal hydrotalcite Portlandite hemicarbonate monocarbonate Amor-Sl FeOOHmic Gbs Mag "
                             "C2S C3A C3S C4AF Lim Gp Brc K2SO4 K2O Na2SO4 Na2O")

# Define solid phases
solidphase_C3AFS084H  = SolidPhase("C3FS0.84H4.32 C3AFS0.84H4.32") # AlFeSi-hydrogarnet_ss
solidphase_ettringite = SolidPhase("ettringite ettringite30") # Ettrignite_ss
solidphase_OH_SO4_AFm = SolidPhase("C4AH13 monosulphate12") # Monosulfate_ss
solidphase_CSHQ       = SolidPhase("CSHQ-TobD CSHQ-TobH CSHQ-JenH CSHQ-JenD KSiOH NaSiOH") # CSH_ss

# Define chemical system by providing database, aqueous phase, minerals, and solid solutions
system = ChemicalSystem(db, solution, minerals,
                            solidphase_C3AFS084H,
                            solidphase_ettringite,
                            solidphase_OH_SO4_AFm,
                            solidphase_CSHQ)

Next, we set up equilibrium specifications, equilibrium conditions, and equilibrium solver all to be used for the
equilibrium calculations:

In [ ]:
# Specify conditions to be satisfied at chemical equilibrium
specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()

# Define temperature and pressure
T = 20.0 # in Celsius
P = 1.0 # in bar

# Define conditions to be satisfied at chemical equilibrium
conditions = EquilibriumConditions(specs)
conditions.temperature(T, "celsius")
conditions.pressure(P, "bar")

# Define equilibrium solver
solver = EquilibriumSolver(specs)

Below, we assemble the chemical state in the proportion water/binder = 0.4, i.e., we assume 40 g of water per 100 g of
the cement clinker presented by the several clinker phases with additives. We run equilibrium calculations and create
chemical and aqueous properties instances based on the obtained chemical state.

In [ ]:
# Define initial equilibrium state
state = ChemicalState(system)
state.set("H2O@", 40, "g") # water
# clinker phases
state.set("C2S" ,  9.70, "g") # belite
state.set("C3A" ,  7.72, "g") # aluminate
state.set("C3S" , 67.31, "g") # alite
state.set("C4AF",  8.14, "g") # ferrite, (CaO)4(Al2O3)(Fe|3|2O3)
# additional
state.set("Gp"    , 3.13, "g") # gypsum, CaSO4(H2O)2
state.set("Cal"   , 0.10, "g") # calcite, CaCO3
state.set("Lim"   , 0.93, "g") # lime, CaO
state.set("Brc"   , 1.31, "g") # brucite, Mg(OH)2
state.set("K2SO4" , 1.34, "g") # potasium-sulfate
state.set("K2O"   , 0.05, "g") # potasium oxide
state.set("Na2SO4", 0.21, "g") # sodium sulfate
state.set("Na2O"  , 0.05, "g") # sodium oxide
# additional oxygen to stabilize the system
state.set("O2@"   , 0.15, "g") # oxygen

# Equilibrate the initial state with given conditions and component amounts
res = solver.solve(state, conditions)
print("res (cemdata18) = ", res.optima.succeeded)

# Output the chemical state to a console
print(state)

# Create chemical properties instance and output it to a console
props = ChemicalProps (state)
print(props)

# Create aqueous properties instance and output it to a console
aprops = AqueousProps (state)
print(aprops)